In [2]:
import boto3
import sagemaker
import pandas as pd
from sklearn.model_selection import train_test_split

region = "us-west-1"
sm_boto3 = boto3.client("sagemaker", region_name=region)
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'shabubucket1'
print("Bucket "+ bucket)

[01/27/25 13:19:28] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=515812;file://e:\AWS\SAGE_MAKER\Mobile_Price_Classification\venv\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=871062;file://e:\AWS\SAGE_MAKER\Mobile_Price_Classification\venv\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

Bucket shabubucket1


In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("iabhishekofficial/mobile-price-classification")

print("Path to dataset files:", path)

e:\AWS\SAGE_MAKER\Mobile_Price_Classification\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 70.6k/70.6k [00:00<00:00, 843kB/s]

Extracting model files...
Path to dataset files: C:\Users\shabn\.cache\kagglehub\datasets\iabhishekofficial\mobile-price-classification\versions\1


In [6]:
df = pd.read_csv(r"mobile_price_classification\train.csv")

In [7]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [9]:
df.shape

(2000, 21)

In [10]:
df['price_range'].value_counts(normalize=True)

price_range
1    0.25
2    0.25
3    0.25
0    0.25
Name: proportion, dtype: float64

In [11]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [13]:
df.isnull().mean()*100 #find the % of missing values

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [14]:
features = list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [15]:
#remove last feature
label = features.pop(-1)
label

'price_range'

In [16]:
x = df[features]
y = df[label]


In [17]:
x.head() # Independent feature

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [18]:
#dependent feature
y.head() #{0:Low_Risk, 1:High_Risk}

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [19]:
print(x.shape)
print(y.shape)

(2000, 20)
(2000,)


In [20]:
y.value_counts() 

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state = 0)

In [22]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(1700, 20)
(300, 20)
(1700,)
(300,)


In [24]:
#convert to DataFrame
trainX = pd.DataFrame(x_train)
trainX[label] = y_train

testX = pd.DataFrame(x_test)
testX[label] = y_test

In [25]:
print(trainX.shape)
print(testX.shape)

(1700, 21)
(300, 21)


In [26]:
trainX.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
1452,1450,0,2.1,0,1,0,31,0.6,114,5,...,1573,1639,794,11,5,9,0,1,1,1
1044,1218,1,2.8,1,3,0,39,0.8,150,7,...,1122,1746,1667,10,0,12,0,0,0,1
1279,1602,0,0.6,0,12,0,58,0.4,170,1,...,1259,1746,3622,17,2,17,0,1,1,3
674,1034,0,2.6,1,2,1,45,0.3,190,3,...,182,1293,969,15,1,7,1,0,0,0
1200,530,0,2.4,0,1,0,32,0.3,88,6,...,48,1012,959,17,7,6,0,1,0,0


In [27]:
trainX.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [28]:
testX.isnull().sum() # test data has no missing values

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [29]:
#saving the dataframes to csv files
trainX.to_csv("train-V-1.csv", index=False)
testX.to_csv("test-V-1.csv", index=False)

In [30]:
#push both files to S3 bucket
sk_prefix = "sagemaker/mobile_price_classification/sklearncontainer"
trainpath = sess.upload_data(path="train-V-1.csv", bucket=bucket, key_prefix=sk_prefix)

testpath = sess.upload_data(path="test-V-1.csv", bucket=bucket, key_prefix=sk_prefix)

In [31]:
print(testpath)
print(trainpath)

s3://shabubucket1/sagemaker/mobile_price_classification/sklearncontainer/test-V-1.csv
s3://shabubucket1/sagemaker/mobile_price_classification/sklearncontainer/train-V-1.csv


In [37]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score
import sklearn
import pandas as pd
import numpy as np
import joblib
import boto3
import pathlib
from io import StringIO
import argparse
import os

#loading the model
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

if __name__ == "__main__":
    print("[INFO]Extracting arguments")
    parser = argparse.ArgumentParser() #argument parser
    
    #wE ARE Training the model in sagemaker
    #By default there are some argument required to train the model
    #so we pass those arguments as command line arguments
    #hyperparameters sent by the client are passed as command line arguments to the model
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    #Data, model and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V--1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")

    args, _ = parser.parse_known_args()
    print("SKlearn Version: ", sklearn._version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO]    READING DATA")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))
    
    features = list(train_df.columns)
    label = features.pop(-1)

    print("Building training and testing dataset")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print("Column order")
    print(features)
    print()

    print("LAbel colum is ",label)
    print()

    print("Data Shape: ")
    print()
    print("----Shape of the training data(85%)----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("----Shape of the testing data(15%)----") 
    print(X_test.shape)
    print(y_test.shape)
    print()

    print("[INFO] Training the Random Forest model...")
    print()
    model = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state)
    model.fit(X_train, y_train)
    print()

    model_path = os.path.join(args.model_dir," model.joblib")
    joblib.dump(model, model_path)
    print()

    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred_test)
    test_rep = classification_report(y_test, y_pred_test)
    print()

    print("-----METRICS RESULST FOR THE TESTING DATA -----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print("[TESTING] Model Accuracy is : ", test_acc)
    print("[TESTING] Classification Report is : \n", test_rep)
    



Writing script.py


In [ ]:
from sagemaker.sklearn.models import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role="arn:",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0
    },
    use_spot_instances=True,
    max_wait=7200,
    max_run=3600)

    


In [ ]:
#launch training job, with asynchronous call, it will create an instance in AWS sagemakermand run
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)


In [ ]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(TrainingJobName=sklearn_estimator.latest_training_job.name)["ModelArtifacts"]["S3ModelArtifacts"]
#where the model artifact is stored
print(f"Model artifact persisted at: " +artifact)


In [ ]:
artifact

In [ ]:
#Copy model For Deployment job to "Custom-sklearn-model"
from sagemaker.sklearn.model import SKLearnModel
from time import time, strftime, gmtime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name = model_name,
    model_data=artifact,
    role="arn:",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

In [ ]:
model_name #returns folder location in S3 bucket

In [ ]:
#MOdel Deployment
endpoint_name = "Custom-sklearn-endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name
)

In [ ]:
predictor

In [ ]:
endpoint_name

In [38]:
testX[features][0:2].values.tolist()

[[1454.0,
  1.0,
  0.5,
  1.0,
  1.0,
  0.0,
  34.0,
  0.7,
  83.0,
  4.0,
  3.0,
  250.0,
  1033.0,
  3419.0,
  7.0,
  5.0,
  5.0,
  1.0,
  1.0,
  0.0],
 [1092.0,
  1.0,
  0.5,
  1.0,
  10.0,
  0.0,
  11.0,
  0.5,
  167.0,
  3.0,
  14.0,
  468.0,
  571.0,
  737.0,
  14.0,
  4.0,
  11.0,
  0.0,
  1.0,
  0.0]]

In [ ]:
print(predictor.predict(testX[features][0:2].values.tolist()))

In [ ]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)